In [ ]:
Install required packages in terminal:

#--Prophet
./scripts/install_package_python2.sh numpy --upgrade
./scripts/install_package_python2.sh fbprophet
./scripts/install_package_python2.sh py4j --upgrade

#--GSheet
./scripts/install_package_python2.sh gspread
./scripts/install_package_python2.sh oauth2client --upgrade
./scripts/install_package_python2.sh pycrypto

#--Styling
./scripts/install_package_python2.sh hide_code

# <span class="mark">City Trip Forecasts w/ FBProphet</span>

# <span class="mark">Setup</span>

In [198]:
#--Prophet
from fbprophet import Prophet #prophet

#--GSheet
import json
import gspread
from oauth2client.service_account import ServiceAccountCredentials

#--Query Tool & Data
import pandas as pd #pandas

#--Charting
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
%matplotlib inline
plt.style.use('fivethirtyeight')

import datetime

# <span class="mark">Queries</span>

In [199]:
qr = Client(user_email=XXX)

In [200]:
query01 = """"""

In [201]:
data01 = qr.execute('warehouse', query01) #run query
data02 = qr.execute('warehouse', query02) #run query

06/23/2017 10:53:37 AM  Created execution 5632b1c7-5cfb-4b34-b621-d8606e89ffc9 
06/23/2017 10:54:46 AM  Query finished. 
06/23/2017 10:54:47 AM  Created execution d55633d2-cd6d-41f8-8966-ad7a15f0b380 
06/23/2017 10:55:48 AM  Query finished. 


In [202]:
df1 = pd.DataFrame(data01.load_data()) #load data into df
df2 = pd.DataFrame(data02.load_data()) #load data into df

In [218]:
trips1 = df1.pivot("day","city_id","trips") #pivot trips to set cities to columns
trips1.reset_index(inplace=True) #collapse index
trips2 = df2
trips2.columns=['day','rouk'] #rename column appropriately
trips3 = pd.merge(trips1, trips2, on='day', how='left') #join data

## set date to forecast from

In [219]:
date_from = '2017-06-18' #select the date to forecast from (pick a sunday)
#2017-05-28

# find the index
for i in range(0,len(trips3['day'])):
    if trips3['day'][i] == date_from:
        lim = i

# trim df
try:
    trips = trips3[:lim]
    del lim
    print 'trimmed at '+date_from
except:
    print 'no trim'

trimmed at 2017-06-18



# <span class="mark">Prohpet</span>

In [220]:
for i in range(1,len(trips.columns)):
    df=trips[['day',list(trips.columns)[i]]] #setup dataframe format
    df.columns=['ds','y'] #rename columns

    m = Prophet() # setup model
    m.fit(df) # fit model
    future = m.make_future_dataframe(periods=365) #setup forecast periods
    forecast = m.predict(future) #generate forecast

    fcst = forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]
    fcst.columns=['day',"fcst_"+str(list(trips.columns)[i]), "fcst_"+str(list(trips.columns)[i])+"_low", "fcst_"+str(list(trips.columns)[i])+"_upp"]
    if i==1:
        fcst_trips=fcst
    else:
        fcst_trips = pd.merge(fcst_trips, fcst, on='day', how='left')

Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
Disabling yearly seas

In [221]:
fcst_trips=fcst_trips.set_index('day') #set idex to be date

# remove forecast values for dates prioir to forcast date
for k in range(0,len(list(fcst_trips.columns))):
    fcst_trips.loc['2015-01-01':date_from, list(fcst_trips.columns)[k]] = np.nan

In [222]:
trips=trips3 #get historical data
trips['day']=pd.to_datetime(trips['day']) # create datetime dates
trips=trips.set_index('day') #index it on time
del trips.index.name
fcst_trips = pd.concat([fcst_trips, trips], axis=1) #join data to commbine history+forecasts

In [223]:
fcst_weekly =fcst_trips.resample('W').sum() #resample weekly
fcst_trips['date']=fcst_trips.index #add date column for saving
fcst_weekly['date']=fcst_weekly.index #add date column for saving

In [224]:
fcst_trips = fcst_trips.replace(np.nan, '', regex=True) #replace NaN with blank
fcst_weekly = fcst_weekly.replace(np.nan, '', regex=True) #replace NaN with blank

In [225]:
#fcst_weekly.describe().transpose()
#fcst_trips.describe().transpose()

# <span class="mark">Push to Google Sheets</span>

In [226]:
# dont forget to share sheet with this email: XXX.iam.gserviceaccount.com
# guide: http://alexsavio.github.io/gspread_oauth2client_intro.html

json_key = 'XXX.json'
scope = ['https://spreadsheets.google.com/feeds']
credentials = ServiceAccountCredentials.from_json_keyfile_name(json_key, scope)
gc = gspread.authorize(credentials)


INFO:oauth2client.client:Refreshing access_token


In [227]:
wks1 = gc.open("00 forecasting").worksheet('Daily')
wks2 = gc.open("00 forecasting").worksheet('Weekly')

In [228]:
#show data from / to dates:
show_from = '2017-01-01'
show_to = '2018-04-01'

d1=fcst_trips[show_from :show_to]
d2=fcst_weekly[show_from :show_to]

In [229]:
all_wks=[wks1,wks2]
all_daf=[d1,d2]

In [230]:
# Define function to be used
def numberToLetters(q):
    q = q - 1
    result = ''
    while q >= 0:
        remain = q % 26
        result = chr(remain+65) + result;
        q = q//26 - 1
    return result

In [231]:
for q in range(0,len(all_wks)):

    daf = all_daf[q] # df to be saved
    wks = all_wks[q] # worksheet to save it in
    
    # Label Columns
    columns = daf.columns.values.tolist() # columns names
    cell_list = wks.range('A1:'+numberToLetters(len(columns))+'1') # selection of the range that will be updated
    # modifying the values in the range
    for cell in cell_list:
        val = columns[cell.col-1]
        if type(val) is str:
            val = val.decode('utf-8')
        cell.value = val
    wks.update_cells(cell_list) # update in batch

    # Fill Data

    num_lines, num_columns = daf.shape # number of lines and columns
    cell_list = wks.range('A2:'+numberToLetters(num_columns)+str(num_lines+1)) # selection of the range that will be updated
    # modifying the values in the range
    for cell in cell_list:
        val = daf.iloc[cell.row-2,cell.col-1]
        if type(val) is str:
            val = val.decode('utf-8')
        elif str(val)=='nan':
            val = 'NaN'
        elif isinstance(val, (int, long, float, complex)):
            #val = round(val) # note that we round all numbers
            val = int(round(val)) # note that we round all numbers
        cell.value = val
    wks.update_cells(cell_list) # update in batch